In [ ]:
import qrcode
from pyzbar import pyzbar
from PIL import Image
import serial
import cv2
from time import time , ctime



# Register a client

In [ ]:
from unittest import result
import pandas as pd
import random
import string

global database
database = pd.DataFrame(
    {
        "Identifier":[],
        "name": [],
        "surname": [],
        "balance": []
    })

def getIdentifier(length):
    # With combination of lower and upper case
    
    result_str = ''.join(random.choice(string.ascii_letters) for i in range(length))
    while(True):
        if len(database[database["Identifier"].str.contains(result_str)]) != 0:
            result_str = ''.join(random.choice(string.ascii_letters) for i in range(length))
        else:
            break
    # print random string
    return result_str


def addCustomer(name , surname,balance):
    Identifier = getIdentifier(13)
    database.append( {
            "Identifier":Identifier,
            "Name": name,
            "surname": surname,
            "Balance": balance
            })


In [ ]:
addCustomer('tiyani','mabunda','150')

In [ ]:


data = {
  "Identifier":['user1',],
  "name": [420, 380, 390],
  "surname": [50, 40, 45],
}

#load data into a DataFrame object:
df = pd.DataFrame(data)

print(df) 

In [ ]:
#initialize qr object
def SaveQrCode(data): 
    qr = qrcode.QRCode(
        version =1,
        box_size =10,
        border=6)


    #add data to qr code
    qr.add_data(data)
    qr.make(fit=True)

    #create an image of qr code
    image = qr.make_image(fill_color="black", back_color= "white")

    #save on the local machine
    image.save("TGBqrcode.png")
    print("QR code has been generated successfully!")

In [ ]:
def ReadQrCode():
    #load qr code imge
    image = Image.open("TGBqrcode.png") 

    qr_code = pyzbar.decode(image)[0]

    #convert into string
    data= qr_code.data.decode("utf-8")
    type = qr_code.type
    text = f"{type}-->, {data}"
    balance = text.split(",")[2]
    SaveQrCode("car1,"+str(int(balance)-10))
    image.close()
   
    print("New balance = R"+balance)
    print("----")

In [ ]:
Price = 5 # will differ by how much a customer spend in the parking lot

In [ ]:
conectionToArduino = serial.Serial('com6',9600) 
 
while (True) :
    if(conectionToArduino.inWaiting()>0):
        instructions = conectionToArduino.readline()
        # camera section 

        loop = True # ftor controlling web cam
        cam = cv2.VideoCapture(0)
        scanner = cv2.QRCodeDetector()

        while loop:
            _,img = cam.read()
            data,_,_ = scanner.detectAndDecode(img)
            cv2.imshow("QRCODE",img)
            cv2.waitKey(500)
            if data :
                print(str(data))
                name, balance, status , timeTrack  = str(data).split(",")

                numeric_balance = int(balance)
                ##SaveQrCode(0)
                loop = False
                if(int(status) == 1):
                    timeSpent= abs(time()-int(timeTrack) )
                    print("time spent = "+str(timeSpent)+"seconds")
                    if timeSpent < 5:
                        numeric_balance = int(numeric_balance) - Price
                    elif timeSpent >= 5 and timeSpent <10:
                        numeric_balance = int(numeric_balance) -( Price*2)
                    else:
                        numeric_balance = numeric_balance - (Price*3)

                    SaveQrCode(name+","+str(numeric_balance)+",0,0")
                    ReadQrCode()
                    
                else:
                    SaveQrCode(name+","+str(int(numeric_balance))+",1,"+str(round(time())))

                break 
            
        if cv2.waitKey(1) & 0xFF == 27:
            break
cv2.destroyAllWindows()
        
        
     